In [3]:
import numpy as np
from PIL import Image, ImageDraw
import sys
import os
HOME_DIR = r'c:\Users\jordy\Documents\python\projects\GenAI_TCG'
sys.path.append(HOME_DIR)
from lib.artdesign import ArtDesignClient

art = ArtDesignClient()
im_size = (1500, 1500)

## 1. generate Biomes images

In [ ]:
biomes = [
    # "OC",
    # "MO",
    # "DE",
    "JU"
]

main_prompts = [
    # "the inside of a wavy dark-blue to light-blue ocean with hundreds of colorful fishs swimming around some detailed coral reefs, algaes", # Ocean [OC]
    # "A huge series of mountains with a snowy peaks, add a lot of small details", # Mountain [MO]
    # "A vast desert with rolling sand dunes, a few scattered cacti, and a bright sun in the sky", # Desert [DE]
    "A dense jungle with tall trees, thick foliage, and a variety of exotic plants and animals", # Jungle [JU]
]
art_styles = "cozy soft pastel colors, gentle lighting, digital painting, delicate details, realistic, high quality"
negative_prompts = ""

prompts = []
for i, p in enumerate(main_prompts):
    prompts.append((p + art_styles, negative_prompts))

for i, prompt in enumerate(prompts):
    # Generate image
    image = art.run_prompts([prompt], size=im_size, im_number=1, shift=3.10, steps=30, cfg=5)

    # save image
    output_filename = os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", f"biome_{biomes[i]}_earth.png")
    image[0].save(output_filename)
    image[0].show()

## 2. Create Earth

combine them in 4 quarters

In [ ]:
img_OC = Image.open(os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", "biome_OC_earth.png"))
img_MO = Image.open(os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", "biome_MO_earth.png"))
img_DE = Image.open(os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", "biome_DE_earth.png"))
img_JU = Image.open(os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", "biome_JU_earth.png"))

# Create a new blank image to hold the quarters
final_img = Image.new('RGB', (im_size[0]*2, im_size[1]*2), color=(0, 0, 0))

biomes_cfg1 = ["OC", "MO", "DE", "JU"]
biomes_cfg2 = ["JU", "DE", "MO", "OC"]
biomes_cfg3 = ["DE", "JU", "OC", "MO"]
biomes_cfg4 = ["MO", "OC", "JU", "DE"]

# Choose which config to use
biomes_cfg = biomes_cfg1  # or biomes_cfg2, biomes_cfg3, biomes_cfg4

# Map biome code to image
biome_images = {
    "OC": img_OC,
    "MO": img_MO,
    "DE": img_DE,
    "JU": img_JU
}

# Paste images according to config: [bottom-left, top-left, top-right, bottom-right]
final_img.paste(biome_images[biomes_cfg[0]], (0, im_size[1]))            # bottom-left
final_img.paste(biome_images[biomes_cfg[1]], (0, 0))                     # top-left
final_img.paste(biome_images[biomes_cfg[2]], (im_size[0], 0))            # top-right
final_img.paste(biome_images[biomes_cfg[3]], (im_size[0], im_size[1]))   # bottom-right

final_img.show()

make it a plain circle (transp ext)

In [12]:
width, height = final_img.size

# Create a circular mask
mask = Image.new('L', (width, height), 0)
draw_mask = ImageDraw.Draw(mask)
center = (width // 2, height // 2)
radius = min(width, height) // 2
draw_mask.ellipse(
    (center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius),
    fill=255
)

# Convert final_img to RGBA if not already
final_img_rgba = final_img.convert("RGBA")

# Apply the mask to the alpha channel
final_img_rgba.putalpha(mask)

output_final_filename = os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", f"biome_earth_nolines.png")
final_img_rgba.save(output_final_filename)
final_img_rgba.show()

draw quarter lines

In [63]:
# Draw a horizontal black line of width 2 through the center
final_img_rgba_with_lines = final_img_rgba.copy()
draw_lines = ImageDraw.Draw(final_img_rgba_with_lines)

# Horizontal line
# Use line width as 1% of the image width
line_width = int(0.4/100 * width)
draw_lines.line([(0, center[1]), (width, center[1])], fill=(0, 0, 0, 255), width=line_width)
# Vertical line
draw_lines.line([(center[0], 0), (center[0], height)], fill=(0, 0, 0, 255), width=line_width)

# final_img_rgba_with_lines.show()

then divide each quarter by a certain number

In [64]:
# Number of divisions per quarter (like minute marks on a watch)
divisions_per_quarter = 6

# Draw short lines only on the exterior of the circle
final_img_with_watch_lines = final_img_rgba_with_lines.copy()
draw_watch = ImageDraw.Draw(final_img_with_watch_lines)

total_divisions = 4 * divisions_per_quarter
line_length = int(0.10 * radius)  # Length of the exterior tick marks is 10% of the radius

for i in range(total_divisions):
    angle = (np.pi / 2) * (i / divisions_per_quarter)
    # Start point: just inside the circle
    x_start = int(center[0] + (radius - line_length) * np.cos(angle))
    y_start = int(center[1] + (radius - line_length) * np.sin(angle))
    # End point: on the circle edge
    x_end = int(center[0] + radius * np.cos(angle))
    y_end = int(center[1] + radius * np.sin(angle))
    ticks_width = int(0.2/100 * width)
    draw_watch.line([(x_start, y_start), (x_end, y_end)], fill=(0, 0, 0, 255), width=ticks_width)

final_img_with_watch_lines.show()

output_filename = os.path.join(HOME_DIR, "lib", "artdesign", "cards_assets", f"earth_cgf4.png")
final_img_with_watch_lines.save(output_filename)

first test

In [ ]:
img = Image.new('RGB', (624, 624), color=(173, 216, 230))  # light blue RGB
draw = ImageDraw.Draw(img)
width, height = img.size

# Draw several sine wave motifs across the image
for i in range(0, height, 60):
    points = [
        (x, int(i + 20 * np.sin(2 * np.pi * (x / 100) + i)))
        for x in range(width)
    ]
    draw.line(points, fill=(0, 105, 148), width=3)
img.save('blue.png')

img_yellow = Image.new('RGB', (624, 624), color=(255, 255, 153))  # light yellow RGB
draw_yellow = ImageDraw.Draw(img_yellow)

for i in range(0, height, 60):
    points = [
        (x, int(i + 20 * np.sin(2 * np.pi * (x / 100) + i)))
        for x in range(width)
    ]
    draw_yellow.line(points, fill=(255, 140, 0), width=3)  # orange RGB

img_yellow.save('yellow.png')

img_brown = Image.new('RGB', (624, 624), color=(139, 69, 19))  # brown RGB
draw_brown = ImageDraw.Draw(img_brown)

# Draw sine wave motifs in a lighter brown
for i in range(0, height, 60):
    points = [
        (x, int(i + 20 * np.sin(2 * np.pi * (x / 100) + i)))
        for x in range(width)
    ]
    draw_brown.line(points, fill=(205, 133, 63), width=3)  # peru RGB

# Draw white triangles at regular intervals
triangle_size = 40
for y in range(0, height, 120):
    for x in range(0, width, 120):
        triangle = [
            (x + triangle_size // 2, y),  # top
            (x, y + triangle_size),       # bottom left
            (x + triangle_size, y + triangle_size)  # bottom right
        ]
        draw_brown.polygon(triangle, fill=(255, 255, 255))

img_brown.save('brown.png')

img_green = Image.new('RGB', (624, 624), color=(144, 238, 144))  # light green RGB
draw_green = ImageDraw.Draw(img_green)

# Draw several circle motifs across the image
circle_radius = 20
for y in range(0, height, 80):
    for x in range(0, width, 80):
        left_up = (x + 40 - circle_radius, y + 40 - circle_radius)
        right_down = (x + 40 + circle_radius, y + 40 + circle_radius)
        draw_green.ellipse([left_up, right_down], outline=(34, 139, 34), width=4)  # forest green

img_green.save('green.png')